In [ ]:
# pip install timm

In [1]:
import torch
from torch import nn
import torchvision
from torchvision import transforms
from torch.utils.data import Dataset
import timm

from PIL import Image
import os
from torch.utils.data import DataLoader

from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
from sklearn.metrics import f1_score
import copy

import time
from tqdm import tqdm


# Функция для установки флага requires_grad для всех параметров модели
def set_requires_grad(model, value=False):
    for param in model.parameters():
        param.requires_grad = value

# Функция для обучения модели
def train_model(model, dataloaders, criterion, optimizer, phases, num_epochs=3):
    start_time = time.time()

    f1_history = {k: list() for k in phases}
    loss_history = {k: list() for k in phases}
    best_f1 = 0.0  # Для отслеживания лучшего значения F1
    best_model_wts = copy.deepcopy(model.state_dict())

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in phases:
            if phase == 'train':
                model.train()  
            else:
                model.eval()  

            running_loss = 0.0
            all_preds = []
            all_labels = []

            # # Итерация по данным
            n_batches = len(dataloaders[phase])
            for inputs, labels in tqdm(dataloaders[phase], total=n_batches):
                inputs = inputs.to(device)
                labels = labels.to(device)

                optimizer.zero_grad()

                # Прямой проход
                # Отслеживание истории градиентов только в фазе обучения
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    loss = criterion(outputs, labels)

                    _, preds = torch.max(outputs, 1)

                    # Обратный проход и оптимизация только в фазе обучения
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # статистики
                running_loss += loss.item() * inputs.size(0)
                all_preds.extend(preds.cpu().numpy())
                all_labels.extend(labels.cpu().numpy())

            epoch_loss = running_loss / len(dataloaders[phase].dataset)

            # Вычисление F1-меры
            epoch_f1 = f1_score(all_labels, all_preds, average='macro')
            f1_history[phase].append(epoch_f1)

            print('{} Loss: {:.4f} F1: {:.4f}'.format(phase, epoch_loss, epoch_f1))
            loss_history[phase].append(epoch_loss)

            # Обновляем лучшее значение F1 и веса модели, если находим новое лучшее значение
            if phase == 'val' and epoch_f1 > best_f1:
                best_f1 = epoch_f1
                best_model_wts = copy.deepcopy(model.state_dict())
        print()
    time_elapsed = time.time() - start_time
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60,
                                                        time_elapsed % 60))
    model.load_state_dict(best_model_wts)
    return model, loss_history, f1_history

# Функция для инициализации модели
def init_model(device, num_classes):
    model = timm.create_model('convnext_base', pretrained=True)
    in_features = model.head.fc.in_features 
    model.head.fc = nn.Linear(in_features, num_classes)  
    model = model.to(device)
    return model

# Класс датасета для работы с изображениями
class ArtDataset(Dataset):
    def __init__(self, root_dir, csv_path=None, transform=None):

        self.transform = transform
        self.files = [os.path.join(root_dir, fname) for fname in os.listdir(root_dir)]
        self.targets = None
        if csv_path:
            df = pd.read_csv(csv_path, sep="\t")
            self.targets = df["label_id"].tolist()
            self.files = [os.path.join(root_dir, fname) for fname in df["image_name"].tolist()]

    def __len__(self):
        return len(self.files)

    def __getitem__(self, idx):
        image = Image.open(self.files[idx]).convert('RGB')
        target = self.targets[idx] if self.targets else -1
        if self.transform:
            image = self.transform(image)
        return image, target


MODEL_WEIGHTS = "./baseline.pt" # в этот файл будут сохранены веса модели из лучшей эпохи
TRAIN_DATASET = "./ваша_директория/train/"
TRAIN_CSV = "./ваша_директория/train.csv"

#Запуск основной программы
if __name__ == "__main__":
    # Задаем размер изображения и трансформации для предобработки изображений
    img_size = 260
    trans = transforms.Compose([
        transforms.Resize((img_size, img_size)),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ])

    # Создаем датасет
    dset = ArtDataset(TRAIN_DATASET, TRAIN_CSV, trans)
    labels = dset.targets  # Получаем метки классов
    indices = list(range(len(labels)))  # Получаем индексы данных

    # Разделение данных на обучающую и тестовую выборки
    ind_train, ind_test, _, _ = train_test_split(indices, labels, test_size=0.2, random_state=139, stratify=labels)

    # Создание подмножеств данных для обучения и тестирования
    trainset = torch.utils.data.Subset(dset, ind_train)
    testset = torch.utils.data.Subset(dset, ind_test)

    # Параметры загрузки данных
    batch_size = 16
    num_workers = 4

    # Создание загрузчиков данных
    trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=num_workers)
    testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size, shuffle=False, num_workers=num_workers)

    # Словарь с загрузчиками данных для обучения и валидации
    loaders = {'train': trainloader, 'val': testloader}

    # Определение устройства для обучения (GPU или CPU)
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

    # Инициализация модели
    model = init_model(device, num_classes=35)

    # Оптимизаторы для предобучения и полного обучения модели
    pretrain_optimizer = torch.optim.AdamW(model.head.fc.parameters(), lr=0.001, weight_decay=0.01)
    train_optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
    
    # Функция потерь
    criterion = nn.CrossEntropyLoss()

    # Обучение модели на предобученных слоях
    pretrain_results = train_model(model, loaders, criterion, pretrain_optimizer, phases=['train', 'val'], num_epochs=3)

    # Разрешение обучения всех слоев модели
    set_requires_grad(model, True)

    # Обучение модели
    train_results = train_model(model, loaders, criterion, train_optimizer, phases=['train', 'val'], num_epochs=10)

    # Сохранение весов модели
    torch.save(model.state_dict(), MODEL_WEIGHTS)

Epoch 0/2
----------


100%|█████████████████████████████████████████████████████████████████████████████████| 431/431 [08:41<00:00,  1.21s/it]


train Loss: 0.8284 F1: 0.6559


100%|█████████████████████████████████████████████████████████████████████████████████| 108/108 [00:22<00:00,  4.70it/s]


val Loss: 0.5006 F1: 0.7549

Epoch 1/2
----------


...

val Loss: 0.4057 F1: 0.7916

Epoch 9/9
----------


100%|█████████████████████████████████████████████████████████████████████████████████| 431/431 [07:28<00:00,  1.04s/it]


train Loss: 0.0100 F1: 0.9988


100%|█████████████████████████████████████████████████████████████████████████████████| 108/108 [00:23<00:00,  4.67it/s]


val Loss: 0.4079 F1: 0.7984

Training complete in 77m 26s
